<a href="https://colab.research.google.com/github/TalissaMoura/AmazonC4DatasetAugmented/blob/main/notebooks/1_pipeline_build_hard_queries_with_one_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pipeline for build the hard queries

# 2. Functions

In [1]:
!pip install outlines

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.4/98.4 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.8 MB/s eta 0:00:00


In [13]:
import os, json, re, torch
import outlines
from huggingface_hub import login,snapshot_download, upload_file, HfApi
from transformers import AutoModelForCausalLM, AutoTokenizer
from google.colab import userdata
import pandas as pd
import pyarrow as pa
import pyarrow.dataset as ds
import pyarrow.parquet as pq
from pydantic import BaseModel,ValidationError
from transformers import AutoModelForCausalLM, AutoTokenizer


In [22]:
class ProductReviewOutline(BaseModel):
    positive_points: list[str|None]
    negative_points: list[str|None]

def extract_json_from_markdown(response_text: str) -> str:
    """Extract JSON from markdown code blocks"""
    # Look for ```json ... ``` or ``` ... ```
    pattern = r'```(?:json)?\s*(.*?)\s*```'
    matches = re.findall(pattern, response_text, re.DOTALL)

    if matches:
        # Return the first JSON block found
        return matches[0].strip()
    else:
        # If no markdown blocks, return original text
        return response_text

def clean_and_parse_json(response_text: str) -> ProductReviewOutline | None:
    # Step 1: Extract JSON from markdown if present
    json_content = extract_json_from_markdown(response_text)

    # Step 2: Remove any remaining backticks or markdown artifacts
    json_content = json_content.strip()

    # Step 3: Handle potential escaping issues
    if json_content.startswith('"') and json_content.endswith('"'):
        # Unescape if it's still a string representation
        try:
            json_content = json.loads(json_content)
        except json.JSONDecodeError:
            pass  # Keep as is if it's not double-escaped

    # Step 4: Parse and validate
    try:
        if isinstance(json_content, str):
            return ProductReviewOutline.model_validate_json(json_content)
        else:
            return ProductReviewOutline.model_validate(json_content)
    except ValidationError as e:
        print(f"Validation error: {e}")
        return None
    except json.JSONDecodeError as e:
        print(f"JSON decode error: {e}")
        return None

def load_model(model_name):
  if "model" not in globals():
    print("Loading model and tokenizer...")
    tokenizer = AutoTokenizer.from_pretrained(model_name,fix_mistral_regex=True,device_map="cpu",)
    model = AutoModelForCausalLM.from_pretrained(model_name, dtype=torch.float16,device_map="auto")

  # 🔍 Validate placement
  if not next(model.parameters()).is_cuda:
      raise RuntimeError("Model failed to load on GPU!")
  print("✅ Model successfully on GPU.")
  return model,tokenizer


def generate_positive_and_negative_points_for_reviews(df_reviews,model,tokenizer,save_local=True):
  batches = df_reviews.to_batches()
  for i,batch in enumerate(batches):
    # Create the parquet dataset
    new_rows = []
    for row in batch.to_pylist():
      positive_review = row['ori_positive_review_distinct']
      semi_positive_reviews = "\n".join([f"{i} - {row['ori_semi_positive_review_list'][i]}" for i in range(len(row['ori_semi_positive_review_list']))])
      messages = [
          {"role": "system", "content": system_template()},
          {"role": "user", "content": user_template(positive_review=positive_review,semi_positive_reviews=semi_positive_reviews)}
      ]
      text = tokenizer.apply_chat_template(
          messages,
          tokenize=False,
          add_generation_prompt=True
      )

    # Append new column for table
      model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

      generated_ids = model.generate(
          **model_inputs,
          max_new_tokens=2048
      )
      generated_ids = [
          output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
      ]

      response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
      response_treated = clean_and_parse_json(response)
      if response_treated:
        new_rows.append(response_treated)

    batch_pylist = batch.to_pylist()
    new_data = []
    for new_row,data in zip(new_rows,batch_pylist):
        print('new_row-dict:',new_row.__dict__)
        positive_points = new_row.__dict__['positive_points']
        negative_points = new_row.__dict__['negative_points']
        data['positive_points'] = positive_points
        data['negative_points'] = negative_points
        new_data.append(data)

    if save_local:
      batch = pa.Table.from_pylist(new_data)
      pq.write_table(batch, f"/content/datasets/procesed/review_analysis/all_Beauty_review_analysis_part_{i}.parquet")
      print(f"Batch {i} done.")

def generate_hard_negative_queries(df_reviews,model,tokenizer,save_local=True):
  batches = df_reviews.to_batches()
  for i,batch in enumerate(batches):
    # Create the parquet dataset
    new_rows = []
    for row in batch.to_pylist():
      positive_list = "-".join(row['positive_points'])
      negative_list = "-".join(row['negative_points'])
      queries = []
      for _ in range(3):
        messages = [
            {"role": "system", "content": system_template()},
            {"role": "user", "content": user_template(positive_list=positive_list,
                                                      negative_list=negative_list)}
        ]
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )

        ## Append new column for table
        model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=2048
        )
        generated_ids = [
            output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]

        response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
        print(response)
        if len(response)>=100:
          queries.append(response)

      new_rows.append(queries)

    batch_pylist = batch.to_pylist()
    new_data = []
    for new_row,data in zip(new_rows,batch_pylist):
        data["hard_negative_query"] = new_row
        new_data.append(data)
    if save_local:
      batch = pa.Table.from_pylist(new_data)
      pq.write_table(batch, f"/content/datasets/procesed/hard_negative_queries/all_Beauty_part_{i}.parquet")
      print(f"Batch {i} done.")

def save_file_in_remote(local_filepath,remote_filepath):
  upload_file(
      path_or_fileobj=f"{local_filepath}",
      path_in_repo=f"{remote_filepath}",
      repo_id="Talissa/AmazonC4Augmented",
      repo_type="dataset",
      commit_message=f"Add {remote_filepath}"
  )
  print("File saved locally")

In [9]:
# setup hugging faces
from huggingface_hub import login,snapshot_download, upload_file, HfApi
from google.colab import userdata
import os
HF_TOKEN = userdata.get("HF_TOKEN")
login(HF_TOKEN)
hf_api = HfApi(token=HF_TOKEN)

# 3. Load data

In [5]:
from google.colab import userdata
from huggingface_hub import snapshot_download
from datasets import load_dataset
import pandas as pd
hf_token = userdata.get('HF_TOKEN')
df_amazonc4 = load_dataset('McAuley-Lab/Amazon-C4')['test'].data.table
snapshot_download(
    repo_id='Talissa/AmazonC4Augmented',
    repo_type='dataset',
    local_dir='datasets',
    allow_patterns=['raw/review_and_meta_categories/filtered_4_star_reviews_All_Beauty.parquet'],
    token=hf_token,
)

README.md: 0.00B [00:00, ?B/s]

test.csv:   0%|          | 0.00/12.5M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/21223 [00:00<?, ? examples/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

raw/review_and_meta_categories/filtered_(…):   0%|          | 0.00/21.4k [00:00<?, ?B/s]

'/content/datasets'

In [6]:
import pyarrow as pa
import pyarrow.dataset as ds
import pyarrow.parquet as pq

df_filtered_reviews = pq.read_table('/content/datasets/raw/review_and_meta_categories/filtered_4_star_reviews_All_Beauty.parquet')
df_filtered_reviews

pyarrow.Table
ori_review: string
asin: string
rating: double
title: string
features: list<element: string>
  child 0, element: string
description: list<element: string>
  child 0, element: string
----
ori_review: [["Very refreshing, cleans well without over drying my skin. Is not exfoliating at all, and doesn't really lather that great, which is why I didn't give it the full five stars, but works well regardless. It rinsed off my face easily, something I appreciate. A few other cleansers seem to never fully rinse, even with a washcloth, and I hate that. This does what I need it to do without any adverse side affects, and my skin seems to be doing well with it.","Nice  brushes, I love them but my daughter thinks they are pokey. They do help remove tangled hair but still use a finning brush too.","I go to the dentist every three months due to plaque buildup and this tool has helped me to control plaque in between visits.  Very sharp so you do have to be very careful when using it.  Vibra

# 4. Build the pipeline

1 - Fetching data for apply first pipeline for list positive and negative points


In [7]:
import pyarrow as pa

df_all_reviews = df_amazonc4.select(['item_id','ori_review']).rename_columns({'ori_review':'ori_positive_review'}).join(
    df_filtered_reviews.select(['asin','ori_review']).rename_columns({'asin':'item_id','ori_review':'ori_semi_positive_review'}),
    join_type='inner',
    keys=['item_id']
)

df_final = df_all_reviews.group_by("item_id").aggregate([
    ("ori_semi_positive_review", "list"),
    ("ori_positive_review", "distinct")
])
print(df_final)

pyarrow.Table
item_id: string
ori_semi_positive_review_list: list<item: string>
  child 0, item: string
ori_positive_review_distinct: list<item: string>
  child 0, item: string
----
item_id: [["B0B186FXGM","B099KSC4L2","B08MF3W9DZ","B0B34GRFL2","B0B86TKMP8","B0BXXD1FZ4","B08L5KN7X4","B00K7J4S6G","B09TT8ZVPN"]]
ori_semi_positive_review_list: [[["The masks are really good for hydrating skin and moisturizing. They are super easy to use and fit really well. My mother uses this every morning and she highly recommends it.","While the mask was a tiny bit small compared to most brands I have tried, it had plenty of serum and left my skin looking and feeling very hydrated and soothed. I knew it has tea tree oil in it, which I love, but it was slightly too tingly for me to have on for 20min. and I had to take it off a little sooner. It didn’t harm my skin in any way, but I don’t love the tingles.<br />I do love the cellulose material of this mask, it’s so much better for the environment!"],["Whi

In [ ]:
# check if we have at least 1 semi positive review and 1 one positive review for each item
from itertools import groupby
items_list = df_final.to_pylist()
for k,g in groupby(items_list, lambda x: x['item_id']):
  items = list(g)
  print(k,'qty_semi_positive_review:',[len(i['ori_semi_positive_review_list']) for i in items],'qty_positive_review:',[len(i['ori_positive_review_distinct']) for i in items])

B099KSC4L2 qty_semi_positive_review: [1] qty_positive_review: [1]
B0B34GRFL2 qty_semi_positive_review: [3] qty_positive_review: [1]
B0B86TKMP8 qty_semi_positive_review: [2] qty_positive_review: [1]
B0B186FXGM qty_semi_positive_review: [2] qty_positive_review: [1]
B08L5KN7X4 qty_semi_positive_review: [3] qty_positive_review: [1]
B00K7J4S6G qty_semi_positive_review: [1] qty_positive_review: [1]
B09TT8ZVPN qty_semi_positive_review: [3] qty_positive_review: [1]
B08MF3W9DZ qty_semi_positive_review: [2] qty_positive_review: [1]
B0BXXD1FZ4 qty_semi_positive_review: [2] qty_positive_review: [1]


2 - Fetch the llm to build the list of positive points and negative points

In [10]:
import outlines, pathlib

if not pathlib.Path('/content/prompts/').exists():
  pathlib.Path.mkdir('/content/prompts')

try:
  repo_id = "Talissa/AmazonC4Augmented"
  filename = f"prompts/system_prompt_analysis_review"
  if not hf_api.file_exists(repo_id=repo_id,filename=filename,repo_type="dataset"):
    raise FileNotFoundError
except FileNotFoundError:
    system_template = \
    """
    You are a text summarizer. You are given user reviews about a product purchased online, and you need to provide a summary of the product's positive and negative attributes.

    Build two separate lists: one containing the summary about **positive aspects** and one containing the **negative aspects** regarding the product identified across **all** the reviews. Be
    as most descriptive and specific as possible. Focus on concrete features, benefits, and issues mentioned in the reviews.

    Return the results in a JSON format that has the following structure:
    {
      "positive_points": ["a point...", "another point..."],
      "negative_points": ["a point...", "another point..."]
    }
    IMPORTANT: Always complete the JSON structure. If there are no negative points, return an array with an empty string: "negative_points": [].
    Do not truncate your response - always complete the JSON object.
    """


    with open('/content/prompts/system_template_review_analysis.txt', 'w+') as f:
          f.write(system_template)
          upload_file(
              path_or_fileobj=f"/content/prompts/system_template_review_analysis.txt",
              path_in_repo=f"prompts/system_template_review_analysis.txt",
              repo_id="Talissa/AmazonC4Augmented",
              repo_type="dataset",
              commit_message="Add system prompt for review analysis"
          )


In [11]:
try:
  repo_id = "Talissa/AmazonC4Augmented"
  filename = f"prompts/user_prompt_analysis_review"
  if not hf_api.file_exists(repo_id=repo_id,filename=filename,repo_type="dataset"):
    raise FileNotFoundError
except FileNotFoundError:
    user_template = \
    """
    A super satisfied user review - 5 stars -: \n {{positive_review}} \n

    Satisfied user reviews - 4 stars -: \n {{semi_positive_reviews}} \n
    """

    with open('/content/prompts/user_prompt_analysis_review.txt', 'w') as f:
          f.write(user_template)
          upload_file(
              path_or_fileobj=f"/content/prompts/user_prompt_analysis_review.txt",
              path_in_repo=f"prompts/user_prompt_analysis_review.txt",
              repo_id="Talissa/AmazonC4Augmented",
              repo_type="dataset",
              commit_message="Add user prompt for review analysis"
          )


In [14]:
# Model for the response

path_procesed_review_analysis = pathlib.Path('/content/datasets/procesed/review_analysis')

if not pathlib.Path(path_procesed_review_analysis).exists():
  pathlib.Path.mkdir(path_procesed_review_analysis,exist_ok=True,parents=True)

MODEL_NAME = "Qwen/Qwen2.5-3B-Instruct"


system_template = outlines.Template.from_file('/content/prompts/system_template_review_analysis.txt')
user_template = outlines.Template.from_file('/content/prompts/user_prompt_analysis_review.txt')

model,tokenizer = load_model(MODEL_NAME)

generate_positive_and_negative_points_for_reviews(
    df_reviews=df_final,
    model=model,
    tokenizer=tokenizer,
    save_local=True
)



Loading model and tokenizer...


config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

✅ Model successfully on GPU.
new_row-dict: {'positive_points': ['lots of essence', 'refreshing and tingly from tea tree oil', 'skin looked and felt nice afterwards', 'played well with her current skincare routine', 'no redness', 'really good for hydrating skin and moisturizing', 'super easy to use', 'fit really well', 'plenty of serum', 'left my skin looking and feeling very hydrated and soothed', 'cellulose material is better for the environment'], 'negative_points': ['bit small compared to most brands', 'too tingly for some users', "not a perfect fit like some other masks she's gotten in the past"]}
new_row-dict: {'positive_points': ['works well for eyebrows', 'satisfies long-standing issue with eyebrows', 'mixes well with other products (Maddluv)', 'improves appearance of eyebrows'], 'negative_points': ['requires brow hair for adhesion', 'may look artificial without brow hair', 'does not adhere to skin without brow hair']}
new_row-dict: {'positive_points': ['Great brushes for curly 

In [ ]:
# # Delete models and tokenizer
# import gc
# import torch

# # Delete global variables
# if 'model' in globals():
#     del globals()['model']
#     print("✅ Removed 'model' from globals")

# if 'tokenizer' in globals():
#     del globals()['tokenizer']
#     print("✅ Removed 'tokenizer' from globals")

# if 'model_wrapper' in globals():
#     del globals()['model_wrapper']
#     print("✅ Removed 'model_wrapper' from globals")

# if 'generator' in globals():
#     del globals()['generator']
#     print("✅ Removed 'generator' from globals")

# # Force garbage collection to free CPU memory
# gc.collect()

# # Clear GPU cache to free up CUDA memory
# if torch.cuda.is_available():
#     torch.cuda.empty_cache()
#     print("✅ GPU cache cleared")
# else:
#     print("CUDA not available")

# print("✅ Cleanup complete - model/tokenizer removed from CPU/GPU")

✅ GPU cache cleared
✅ Cleanup complete - model/tokenizer removed from CPU/GPU


3 - Read parquet and generate queries for the products

In [15]:
df_batch = pq.read_table('/content/datasets/procesed/review_analysis/all_Beauty_review_analysis_part_0.parquet')
print(df_batch)

pyarrow.Table
item_id: string
ori_semi_positive_review_list: list<element: string>
  child 0, element: string
ori_positive_review_distinct: list<element: string>
  child 0, element: string
positive_points: list<element: string>
  child 0, element: string
negative_points: list<element: string>
  child 0, element: string
----
item_id: [["B0B186FXGM","B099KSC4L2","B08MF3W9DZ","B0B34GRFL2","B0B86TKMP8","B0BXXD1FZ4","B08L5KN7X4","B00K7J4S6G","B09TT8ZVPN"]]
ori_semi_positive_review_list: [[["The masks are really good for hydrating skin and moisturizing. They are super easy to use and fit really well. My mother uses this every morning and she highly recommends it.","While the mask was a tiny bit small compared to most brands I have tried, it had plenty of serum and left my skin looking and feeling very hydrated and soothed. I knew it has tea tree oil in it, which I love, but it was slightly too tingly for me to have on for 20min. and I had to take it off a little sooner. It didn’t harm my ski

In [16]:
if not pathlib.Path('/content/prompts/').exists():
  pathlib.Path.mkdir('/content/prompts')

try:
  repo_id = "Talissa/AmazonC4Augmented"
  filename = f"prompts/system_prompt_generate_hard_negative_query"
  if not hf_api.file_exists(repo_id=repo_id,filename=filename,repo_type="dataset"):
    raise FileNotFoundError
except FileNotFoundError:
    system_template = \
    """
    You are a text generator. You receive two lists: one containing positive aspects and another containing negative aspects of a product purchased online, from the users’ point of view.

    Based on these lists, generate a first-person, natural-language search query (i.e., phrased as if a real person were typing it into a search engine or asking a friend).

    The query should reflect interest in finding a product that containes one of the key positive qualities listed, but avoids or improves upon one specific negative aspect mentioned.

    IMPORTANT:
    - The query must sound conversational and realistic.
    - It must be at least 100 characters long.
    - If no negative aspects are provided (i.e., the negative list is empty), return an empty string ("").
    """


    with open('/content/prompts/system_prompt_generate_hard_negative_query.txt', 'w+') as f:
          f.write(system_template)
          upload_file(
              path_or_fileobj=f"/content/prompts/system_prompt_generate_hard_negative_query.txt",
              path_in_repo=f"prompts/system_prompt_generate_hard_negative_query.txt",
              repo_id="Talissa/AmazonC4Augmented",
              repo_type="dataset",
              commit_message="Add system prompt for generate hard negative queries"
          )


In [17]:
try:
  repo_id = "Talissa/AmazonC4Augmented"
  filename = f"prompts/user_prompt_generate_hard_negative_queries"
  if not hf_api.file_exists(repo_id=repo_id,filename=filename,repo_type="dataset"):
    raise FileNotFoundError
except FileNotFoundError:
    user_template = \
    """
    positive product characteristics -: \n {{positive_list}} \n

    negative product characteristics -: \n {{negative_list}} \n
    """

    with open('/content/prompts/user_prompt_generate_hard_negative_queries.txt', 'w') as f:
          f.write(user_template)
          upload_file(
              path_or_fileobj=f"/content/prompts/user_prompt_generate_hard_negative_queries.txt",
              path_in_repo=f"prompts/user_prompt_generate_hard_negative_queries.txt",
              repo_id="Talissa/AmazonC4Augmented",
              repo_type="dataset",
              commit_message="Add user prompt for generate hard negative queries"
          )


No files have been modified since last commit. Skipping to prevent empty commit.


In [18]:
# Model for the response

path_procesed_hnq = pathlib.Path('/content/datasets/procesed/hard_negative_queries')

if not pathlib.Path(path_procesed_hnq).exists():
  pathlib.Path.mkdir(path_procesed_hnq,exist_ok=True,parents=True)


system_template = outlines.Template.from_file('/content/prompts/system_prompt_generate_hard_negative_query.txt')
user_template = outlines.Template.from_file('/content/prompts/user_prompt_generate_hard_negative_queries.txt')


generate_hard_negative_queries(
    df_batch,
    model,
    tokenizer,
    save_local=True
)


looking for a tea tree oil serum that's great for hydration and soothing, but not too tingling like some others, and also fits well without being too small like this one was
looking for a tea tree oil serum that's super hydrating and moisturizing, but not too tingling like some others - any options that come in a larger size and fit better?
looking for a tea tree oil serum that's super hydrating and moisturizing but not too tingling like this one, also hoping it fits better than the others I've tried
looking for eyebrow product that mixes well with others but doesn't require actual brow hairs for adhesive properties
looking for an eyebrow product that mixes well with others but doesn't require actual brow hairs for adhesion
looking for eyebrow product that mixes well with others but doesn't require actual brow hairs for adhesion
I'm looking for great curly hair brushes that don't need to be too tough, but can still stand up to my daughter's touch. Any suggestions?
looking for curly hai

In [19]:
df_batch_hq = pq.read_table('/content/datasets/procesed/hard_negative_queries/all_Beauty_part_0.parquet')
print(df_batch_hq)

pyarrow.Table
item_id: string
ori_semi_positive_review_list: list<element: string>
  child 0, element: string
ori_positive_review_distinct: list<element: string>
  child 0, element: string
positive_points: list<element: string>
  child 0, element: string
negative_points: list<element: string>
  child 0, element: string
hard_negative_query: list<element: string>
  child 0, element: string
----
item_id: [["B0B186FXGM","B099KSC4L2","B08MF3W9DZ","B0B34GRFL2","B0B86TKMP8","B0BXXD1FZ4","B08L5KN7X4","B00K7J4S6G","B09TT8ZVPN"]]
ori_semi_positive_review_list: [[["The masks are really good for hydrating skin and moisturizing. They are super easy to use and fit really well. My mother uses this every morning and she highly recommends it.","While the mask was a tiny bit small compared to most brands I have tried, it had plenty of serum and left my skin looking and feeling very hydrated and soothed. I knew it has tea tree oil in it, which I love, but it was slightly too tingly for me to have on for 

In [25]:
## Save files in remote

files = ['/content/datasets/procesed/hard_negative_queries/all_Beauty_part_0.parquet',
         '/content/datasets/procesed/review_analysis/all_Beauty_review_analysis_part_0.parquet']
remote_files = ['/datasets/procesed/hard_negative_queries/all_Beauty_part_0.parquet',
                'datasets/procesed/review_analysis/all_Beauty_review_analysis_part_0.parquet']
for local,remote in zip(files,remote_files):
  save_file_in_remote(local,remote)

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...all_Beauty_part_0.parquet: 100%|##########| 20.3kB / 20.3kB            

No files have been modified since last commit. Skipping to prevent empty commit.


File saved locally


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...w_analysis_part_0.parquet: 100%|##########| 16.7kB / 16.7kB            

File saved locally
